# [Travelbrag](http://travelbrag.ca)
Andrea & Brett's travelogue.

In [1]:
import datetime
import pandas

cities = pandas.read_csv("cities.csv")
travelogue = pandas.read_csv("travelogue.csv", parse_dates=["arrived", "departed"])
today = datetime.date.today()
colours = {frozenset(["Andrea", "Brett"]): "#00FA21", frozenset(["Andrea"]): "#F2FA00",
           frozenset(["Brett"]): "#005CFA"}

# What was our last vacation?
A _vacation_ is classified as the last time Andrea and I travelled for non-work purposes together.

In [2]:
both_of_us = travelogue[travelogue["travellers"] == "Andrea & Brett"]
vacations = both_of_us[~both_of_us["work"]]
last_vacation = vacations.sort_values(by="departed").tail(1).iloc[0]
departed = last_vacation["departed"].date()

print("Our last vacation was to {}, {} for {} on {}.".format(last_vacation["city"],
                                                             last_vacation["country"],
                                                             last_vacation["purpose"],
                                                             departed.isoformat()))

Our last vacation was to Los Angeles, USA for Disneyland on 2017-02-23.


# Where have we been in the last 12 months?

In [3]:
import datetime

year_ago = today - datetime.timedelta(days=365)
travelogue[travelogue["arrived"] >= year_ago].sort_values(by="arrived")

,purpose,arrived,departed,travellers,city,country,work,comment
175,Civic Holiday,2016-07-30,2016-08-02,Andrea & Brett,Victoria,CAN,False,NaN
176,Python core sprint,2016-09-04,2016-09-10,Brett,Palo Alto,USA,True,NaN
177,Python core sprint,2016-09-10,2016-09-11,Brett,San Francisco,USA,True,NaN
178,Azure security meetings,2016-10-02,2016-10-05,Brett,Redmond,USA,True,NaN
179,Visit Paul & Gian,2016-10-08,2016-10-13,Andrea & Brett,Kelowna,CAN,False,NaN
180,PyCon CA,2016-11-11,2016-11-14,Brett,Toronto,CAN,True,NaN
181,US Thanksgiving,2016-11-22,2016-11-26,Andrea & Brett,Langley,USA,False,NaN
182,Disneyland,2017-02-16,2017-02-23,Andrea & Brett,Los Angeles,USA,False,"Stayed in Newport Beach and Chino, visited Ana..."


# Where have we visited?

In [4]:
full_country_names = {
    "CAN": "Canada",
    "USA": "America",
    "GBR": "Great Britain",
    "JPN": "Japan",
    "BEL": "Belgium",
    "CZE": "Czech Republic",
    "ESP": "Spain",
    "FRA": "France",
    "CHE": "Switzerland",
    "CUB": "Cuba",
    "ARG": "Argentina",
    "DOM": "Dominican Republic",
}

locations = travelogue[['city', 'country']].drop_duplicates()
countries = locations['country'].drop_duplicates().tolist()
city_names = tuple((row[0], row[1]) for row in locations.values.tolist())

print("(In chronological order of first visit)")
print()
print(len(cities), "cities across", len(countries), "countries:")
for country in countries:
    print("   ", full_country_names[country])
    for city, _ in filter(lambda x: x[1] == country, city_names):
        print("       ", city)

(In chronological order of first visit)

100 cities across 12 countries:
    America
        Yosemite Valley
        Las Vegas
        Anaheim
        Chicago
        Washington, D.C.
        Mammoth Lakes
        Montclair
        New York
        Atlantic City
        Addison
        Rosemont
        Seattle
        Pittsburgh
        Charlottesville
        Langley
        Meridian
        Roseville
        Atlanta
        Raleigh
        San Francisco
        Mountain View
        Monterey
        Portland
        Santa Clara
        Ann Arbor
        Redmond
        Austin
        Union City
        New Haven
        Boise
        Palo Alto
        Los Angeles
    Great Britain
        London
        Brighton
        York
        Edinburgh
        Oxford
        Manchester
        Liverpool
        Ely
        Shoreham-by-Sea
        Birmingham
        Lancaster
        Glasgow
        Stirling
    Japan
        Annaka
        Hiroshima
        Kyoto
        Nara
        Osaka
   

# Is the data valid?
Manually entering data into a spreadsheet is always error-prone, so some quick checks are always useful to catch common mistakes.

In [5]:
import datetime
import re
import urllib.parse

# CITIES
# Latitude
if any(lat < -90 or lat > 90 for lat in cities['latitude']):
    raise ValueError('malformed latitude')
    
# Longitude
if any(lng < -180 or lng > 180 for lng in cities['longitude']):
    raise ValueError('malformed longitude')

# TRAVELOGUE
# Travellers
if any(x not in {"Andrea", "Brett", "Andrea & Brett"} for x in travelogue['travellers']):
    raise ValueError('unrecognized travellers')
    
# City
if any(pandas.isnull(travelogue['city'])):
    raise ValueError('missing the city in the travelogue data')
elif not all(travelogue["city"].isin(cities["city"].values)):
    raise ValueError("city in travelogue but not in cities")

# Country
if any(len(x) != 3 or x.upper() != x for x in travelogue['country']):
    bad_countries = []
    raise ValueError('malformed country')
elif not all(travelogue["country"].isin(cities["country"].values)):
    raise ValueError("country in travelogue but not in cities")
elif len(pandas.merge(travelogue, cities, on=["city", "country"])) != len(travelogue):
    raise ValueError("city/country in travelogue not in cities data")
    
# Arrived/Departed
if any(x.arrived > x.departed for x in travelogue.itertuples()):
    raise ValueError("arrival date passed departure date")
        
print('All travelogue data is valid!')

All travelogue data is valid!


# GeoJSON map output
A GeoJSON file is generated to allow for easy mapping of visited cities.

In [6]:
import json

def split_travellers(travellers):
    return travellers.split(" & ")

complete_travelogue = pandas.merge(travelogue, cities, on=["city", "country"])
geo_data = {}
for trip in complete_travelogue.itertuples():
    location = trip.city, trip.country
    if location not in geo_data:
        data = {"coordinates": [trip.longitude, trip.latitude],
                "travellers": set(split_travellers(trip.travellers)), "last visit": trip.departed,
                "work": bool(trip.work)}
        geo_data[location] = data
    else:
        data = geo_data[location]
        data["travellers"].update(split_travellers(trip.travellers))
        if data["last visit"] < trip.departed:
            data["last visit"] = trip.departed
geojson = {"type": "FeatureCollection", "features": []}
for location, data in geo_data.items():
    point = {"type": "Feature", "geometry": {"type": "Point", "coordinates": data["coordinates"]},
             "properties": {"city": ", ".join(location), "last visit": data["last visit"].strftime("%Y-%m-%d"),
                            "marker-color": colours[frozenset(data["travellers"])], "work": data["work"]}}
    geojson["features"].append(point)
with open("travelogue.geojson", "w", encoding="utf-8") as file:
    json.dump(geojson, file, sort_keys=True)

In [7]:
import folium

world_map = folium.Map(location=[0, 0], tiles='Mapbox Bright', zoom_start=1)
folium.GeoJson(geojson, name="Trips").add_to(world_map)
world_map